# 🧬 UFoE Protein Atlas — 단백질 우주 지도

**단백질 7만 개를 12칸 매트릭스로 분류하여 전체 지도를 그린다**

Author: Young Kang (MCWS) | 2026-02

---

### 좌표계

```
X축: 상극축 (Antagonism) — 곤-건 분리도
Y축: 감 밀도 (Gam Density) — 구조 복잡도  
Z축: 길이 (Length) — 단백질 크기
색상: 클러스터 (C0~C4)
```

### 5유형 = 중심 + 4방위 (건곤감리)

| 클러스터 | 괘 | 핵심 | 감 역할 |
|---------|-----|------|---------|
| C0 효소 | 중심 | 균형 | P(꺾임) → 활성부위 |
| C1 구형 | 곤 | 압축 | 감 적음 → 단순 코어 |
| C2 균형 | 리 | 유동 | G(유연) → 대형 허브 |
| C3 DNA결합 | 건 | 외연 | 감 희소 → 펼쳐진 표면 |
| C4 밀집코어 | 감 | 변곡 | C(고정) → SS-bond 밀집 |

---


## 0. 환경 설정

In [ ]:
!pip install -q requests pandas matplotlib seaborn plotly scikit-learn

import numpy as np
import pandas as pd
import requests
import json
import time
import math
from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import seaborn as sns

print("✅ 패키지 로드 완료")


## 1. UFoE 상수

In [ ]:
# 곤감리건 분류
GON_AAS = set('VILFWMA')   # 7 소수성
GAM_AAS = set('CGP')        # 3 구조적 제약 (결절점)
RI_AAS  = set('STNQHY')    # 6 극성
GEON_AAS = set('KRDE')      # 4 전하

def get_type(aa):
    if aa in GON_AAS: return 'Gon'
    if aa in GAM_AAS: return 'Gam'
    if aa in RI_AAS: return 'Ri'
    if aa in GEON_AAS: return 'Geon'
    return 'Unknown'

TYPES = ['Gon', 'Gam', 'Ri', 'Geon']
ZONES = ['Center', 'Transition', 'Boundary']

# 피보나치 수열
FIB_SET = {1, 2, 3, 5, 8, 13, 21, 34, 55}

def nearest_fibonacci(n):
    fibs = [1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
    if n <= 0: return 1, abs(n-1)
    best = min(fibs, key=lambda f: abs(f-n))
    return best, abs(n-best)

# φ 경계
PHI = (1 + math.sqrt(5)) / 2
CENTER_THRESH = 1 / (PHI ** 2)   # ≈ 0.382
TRANS_THRESH = 1 / PHI            # ≈ 0.618

print("✅ UFoE 상수 정의 완료")
print(f"  곤:감:리:건 = 7:3:6:4")
print(f"  φ 경계: Center < {CENTER_THRESH:.3f}, Transition < {TRANS_THRESH:.3f}")


## 2. RCSB PDB 서열 가져오기

RCSB Search API로 단일 체인 단백질 서열을 가져옵니다.
전체 ~200K 엔트리 중 단백질 체인 ~70K를 대상으로 합니다.


In [ ]:
def fetch_pdb_sequences(max_entries=70000, batch_size=500):
    """RCSB Search API로 단백질 서열 가져오기."""
    
    # RCSB Search API 쿼리: 단백질, X-ray/NMR, 단일체인 우선
    search_url = "https://search.rcsb.org/rcsbsearch/v2/query"
    
    query = {
        "query": {
            "type": "group",
            "logical_operator": "and",
            "nodes": [
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "attribute": "entity_poly.rcsb_entity_polymer_type",
                        "operator": "exact_match",
                        "value": "Protein"
                    }
                },
                {
                    "type": "terminal",
                    "service": "text",
                    "parameters": {
                        "attribute": "entity_poly.pdbx_seq_one_letter_code_can",
                        "operator": "exists"
                    }
                }
            ]
        },
        "return_type": "polymer_entity",
        "request_options": {
            "paginate": {"start": 0, "rows": min(batch_size, max_entries)},
            "results_content_type": ["experimental"],
            "sort": [{"sort_by": "score", "direction": "desc"}]
        }
    }
    
    print(f"🔄 RCSB에서 단백질 서열 가져오는 중...")
    
    all_ids = []
    start = 0
    
    while start < max_entries:
        query["request_options"]["paginate"] = {
            "start": start, 
            "rows": min(batch_size, max_entries - start)
        }
        
        try:
            resp = requests.post(search_url, json=query, timeout=30)
            if resp.status_code != 200:
                print(f"  ⚠ API 오류 ({resp.status_code}) at start={start}")
                break
            
            data = resp.json()
            results = data.get("result_set", [])
            if not results:
                break
            
            for r in results:
                all_ids.append(r["identifier"])
            
            total = data.get("total_count", 0)
            start += len(results)
            
            if start % 5000 == 0:
                print(f"  {start}/{min(total, max_entries)} IDs 수집...")
            
            if len(results) < batch_size:
                break
                
        except Exception as e:
            print(f"  ⚠ 오류: {e}")
            break
        
        time.sleep(0.2)  # Rate limit
    
    print(f"  ✅ {len(all_ids)} polymer entity IDs 수집 완료")
    return all_ids


def fetch_sequences_batch(entity_ids, batch_size=100):
    """GraphQL API로 서열 일괄 가져오기."""
    graphql_url = "https://data.rcsb.org/graphql"
    
    all_data = []
    
    for i in range(0, len(entity_ids), batch_size):
        batch = entity_ids[i:i+batch_size]
        
        # 간단한 REST API로 서열 가져오기
        for eid in batch:
            try:
                url = f"https://data.rcsb.org/rest/v1/core/polymer_entity/{eid}"
                resp = requests.get(url, timeout=10)
                if resp.status_code == 200:
                    data = resp.json()
                    seq = data.get("entity_poly", {}).get("pdbx_seq_one_letter_code_can", "")
                    if seq and len(seq) >= 20:
                        all_data.append({
                            'entity_id': eid,
                            'sequence': seq.replace("\n", ""),
                            'length': len(seq.replace("\n", "")),
                        })
            except:
                pass
        
        if (i + batch_size) % 1000 == 0:
            print(f"  서열 {len(all_data)}개 수집... ({i+batch_size}/{len(entity_ids)})")
        
        time.sleep(0.1)
    
    return all_data


# 더 효율적인 방법: FASTA 다운로드
def fetch_sequences_fasta(max_entries=10000):
    """RCSB FASTA 다운로드로 빠르게 서열 수집."""
    
    # 먼저 ID 목록 가져오기
    entity_ids = fetch_pdb_sequences(max_entries=max_entries)
    
    if not entity_ids:
        print("⚠ ID 수집 실패, 대안 방식 사용...")
        return fetch_sequences_alternative()
    
    print(f"\n🔄 {len(entity_ids)}개 서열 다운로드 중...")
    
    sequences = []
    batch_size = 50
    
    for i in range(0, min(len(entity_ids), max_entries), batch_size):
        batch = entity_ids[i:i+batch_size]
        
        for eid in batch:
            parts = eid.split("_")
            if len(parts) >= 2:
                pdb_id = parts[0]
                entity_num = parts[1]
            else:
                continue
            
            try:
                url = f"https://www.rcsb.org/fasta/entity/{pdb_id}_{entity_num}"
                resp = requests.get(url, timeout=10)
                if resp.status_code == 200:
                    lines = resp.text.strip().split("\n")
                    seq = "".join(l for l in lines if not l.startswith(">"))
                    if len(seq) >= 20 and len(seq) <= 2000:
                        sequences.append({
                            'entity_id': eid,
                            'sequence': seq,
                            'length': len(seq),
                        })
            except:
                pass
        
        if len(sequences) % 500 == 0 and len(sequences) > 0:
            print(f"  {len(sequences)}개 서열 수집 완료...")
        
        time.sleep(0.05)
    
    print(f"\n✅ 총 {len(sequences)}개 서열 수집")
    return sequences


def fetch_sequences_alternative(n=5000):
    """PDB entry 기반 간소화 수집 (백업)."""
    print("  대안: PDB entry 목록 기반 수집...")
    
    # 대표적 PDB ID 범위에서 서열 수집
    search_url = "https://search.rcsb.org/rcsbsearch/v2/query"
    query = {
        "query": {"type": "terminal", "service": "text",
                  "parameters": {"attribute": "rcsb_entry_info.polymer_entity_count_protein",
                                "operator": "greater", "value": 0}},
        "return_type": "entry",
        "request_options": {"paginate": {"start": 0, "rows": n},
                           "sort": [{"sort_by": "rcsb_accession_info.deposit_date", "direction": "desc"}]}
    }
    
    try:
        resp = requests.post(search_url, json=query, timeout=30)
        data = resp.json()
        pdb_ids = [r["identifier"] for r in data.get("result_set", [])]
    except:
        pdb_ids = []
    
    sequences = []
    for pid in pdb_ids[:n]:
        try:
            url = f"https://www.rcsb.org/fasta/entry/{pid}"
            resp = requests.get(url, timeout=10)
            if resp.status_code == 200:
                current_id = ""
                current_seq = []
                for line in resp.text.strip().split("\n"):
                    if line.startswith(">"):
                        if current_seq and current_id:
                            seq = "".join(current_seq)
                            if 20 <= len(seq) <= 2000:
                                sequences.append({'entity_id': current_id, 'sequence': seq, 'length': len(seq)})
                        current_id = f"{pid}_{line.split('|')[0][1:]}" if '|' in line else pid
                        current_seq = []
                    else:
                        current_seq.append(line.strip())
                if current_seq and current_id:
                    seq = "".join(current_seq)
                    if 20 <= len(seq) <= 2000:
                        sequences.append({'entity_id': current_id, 'sequence': seq, 'length': len(seq)})
        except:
            pass
        
        if len(sequences) % 500 == 0 and len(sequences) > 0:
            print(f"  {len(sequences)}개...")
        time.sleep(0.05)
    
    print(f"  ✅ {len(sequences)}개 서열 수집")
    return sequences

print("✅ RCSB 수집 함수 정의 완료")


## 3. 12칸 매트릭스 + 감 분석

In [ ]:
def analyze_sequence(seq):
    """서열 → 12칸 매트릭스 + 감 프로파일 + 상극축."""
    n = len(seq)
    if n < 20:
        return None
    
    # 타입 카운트
    type_counts = defaultdict(int)
    gam_positions = []
    
    for i, aa in enumerate(seq):
        t = get_type(aa)
        if t != 'Unknown':
            type_counts[t] += 1
        if t == 'Gam':
            gam_positions.append(i)
    
    total_known = sum(type_counts.values())
    if total_known < n * 0.8:
        return None
    
    type_ratios = {t: type_counts[t] / total_known for t in TYPES}
    
    # 감 분석
    gam_density = len(gam_positions) / n
    gam_gaps = [gam_positions[i] - gam_positions[i-1] for i in range(1, len(gam_positions))]
    
    near_fib = 0
    mean_gap = 0
    if gam_gaps:
        near_fib = sum(1 for g in gam_gaps if nearest_fibonacci(g)[1] <= 1) / len(gam_gaps)
        mean_gap = np.mean(gam_gaps)
    
    # G, P, C 개별
    g_ratio = sum(1 for aa in seq if aa == 'G') / n
    p_ratio = sum(1 for aa in seq if aa == 'P') / n
    c_ratio = sum(1 for aa in seq if aa == 'C') / n
    
    # 12칸 매트릭스 (서열 위치 기반 Zone)
    # Zone 할당: 정규화된 위치 기준
    matrix = {}
    zone_counts = defaultdict(int)
    zone_type_counts = defaultdict(lambda: defaultdict(int))
    
    for i, aa in enumerate(seq):
        t = get_type(aa)
        if t == 'Unknown':
            continue
        
        # 서열 위치 정규화 (0~1) — 양끝에서 중앙까지
        pos_norm = abs(2 * i / n - 1)  # 0=중앙, 1=양끝
        # 실제로는 접힘 후 3D 위치를 모르므로, 조성 기반으로 근사
        # → 각 1/3 구간의 조성으로 Zone 매트릭스 근사
        if i < n // 3:
            zone = 'Center'
        elif i < 2 * n // 3:
            zone = 'Transition'
        else:
            zone = 'Boundary'
        
        zone_counts[zone] += 1
        zone_type_counts[zone][t] += 1
    
    for z in ZONES:
        z_total = zone_counts[z] if zone_counts[z] > 0 else 1
        for t in TYPES:
            matrix[(z, t)] = zone_type_counts[z][t] / z_total
    
    # 상극축
    antagonism = (
        (matrix.get(('Center','Gon'), 0) - matrix.get(('Boundary','Gon'), 0)) -
        (matrix.get(('Center','Geon'), 0) - matrix.get(('Boundary','Geon'), 0))
    )
    
    # 곤 기울기
    gon_gradient = matrix.get(('Center','Gon'), 0) > matrix.get(('Boundary','Gon'), 0)
    geon_gradient = matrix.get(('Center','Geon'), 0) < matrix.get(('Boundary','Geon'), 0)
    
    # Zone 분포 (전체 대비)
    total_residues = sum(zone_counts.values())
    zone_distribution = {z: zone_counts[z] / total_residues for z in ZONES}
    
    return {
        'length': n,
        'type_ratios': type_ratios,
        'gam_density': gam_density,
        'mean_gap': mean_gap,
        'near_fibonacci': near_fib,
        'g_ratio': g_ratio,
        'p_ratio': p_ratio,
        'c_ratio': c_ratio,
        'antagonism': antagonism,
        'gon_gradient': gon_gradient,
        'geon_gradient': geon_gradient,
        'matrix': matrix,
        'zone_distribution': zone_distribution,
    }

print("✅ 분석 함수 정의 완료")


## 4. 전체 매핑 실행

> 첫 실행은 ~10,000개로 시작합니다. 성공하면 70,000으로 확장하세요.


In [ ]:
# ═══════════════════════════════════════
# 서열 수집 (첫 실행: 10K, 확장: 70K)
# ═══════════════════════════════════════
N_TARGET = 10000  # 첫 실행. 나중에 70000으로 변경

sequences = fetch_sequences_fasta(max_entries=N_TARGET)
print(f"\n수집된 서열: {len(sequences)}개")


In [ ]:
# ═══════════════════════════════════════
# 전체 분석
# ═══════════════════════════════════════
print(f"🔬 {len(sequences)}개 서열 분석 중...")

results = []
for i, entry in enumerate(sequences):
    profile = analyze_sequence(entry['sequence'])
    if profile:
        profile['entity_id'] = entry['entity_id']
        results.append(profile)
    
    if (i + 1) % 2000 == 0:
        print(f"  {i+1}/{len(sequences)} 완료 ({len(results)}개 성공)")

print(f"\n✅ 분석 완료: {len(results)}개 단백질")

# DataFrame 변환
df = pd.DataFrame([{
    'entity_id': r['entity_id'],
    'length': r['length'],
    'antagonism': r['antagonism'],
    'gam_density': r['gam_density'],
    'mean_gap': r['mean_gap'],
    'near_fibonacci': r['near_fibonacci'],
    'gon_ratio': r['type_ratios']['Gon'],
    'gam_ratio': r['type_ratios']['Gam'],
    'ri_ratio': r['type_ratios']['Ri'],
    'geon_ratio': r['type_ratios']['Geon'],
    'g_ratio': r['g_ratio'],
    'p_ratio': r['p_ratio'],
    'c_ratio': r['c_ratio'],
    'gon_gradient': r['gon_gradient'],
    'geon_gradient': r['geon_gradient'],
    'center_gon': r['matrix'].get(('Center','Gon'), 0),
    'boundary_geon': r['matrix'].get(('Boundary','Geon'), 0),
} for r in results])

print(f"\nDataFrame: {df.shape}")
print(df.describe())


## 5. 클러스터링 (K=5)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 클러스터링 피처
features = ['antagonism', 'gam_density', 'gon_ratio', 'geon_ratio', 'mean_gap', 'c_ratio']
X = df[features].fillna(0).values

# 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-Means (5 클러스터 = 건곤감리 + 중심)
kmeans = KMeans(n_clusters=5, random_state=42, n_init=20)
df['cluster'] = kmeans.fit_predict(X_scaled)

# 클러스터별 통계
print("═" * 70)
print("  5개 클러스터 프로파일")
print("═" * 70)

cluster_names = {}
for c in range(5):
    sub = df[df['cluster'] == c]
    mean_antag = sub['antagonism'].mean()
    mean_gam = sub['gam_density'].mean()
    mean_length = sub['length'].mean()
    mean_gap = sub['mean_gap'].mean()
    
    print(f"\n  Cluster {c} (n={len(sub)}):")
    print(f"    길이: {mean_length:.0f} ± {sub['length'].std():.0f}")
    print(f"    상극축: {mean_antag:.4f}")
    print(f"    감 밀도: {mean_gam:.1%}")
    print(f"    감 간격: {mean_gap:.1f}")
    print(f"    곤: {sub['gon_ratio'].mean():.1%}  감: {sub['gam_ratio'].mean():.1%}  "
          f"리: {sub['ri_ratio'].mean():.1%}  건: {sub['geon_ratio'].mean():.1%}")
    print(f"    G: {sub['g_ratio'].mean():.1%}  P: {sub['p_ratio'].mean():.1%}  C: {sub['c_ratio'].mean():.1%}")
    print(f"    피보나치 ±1: {sub['near_fibonacci'].mean():.0%}")


## 6. 단백질 우주 지도 시각화

In [ ]:
# ═══════════════════════════════════════
# 2D 지도: 상극축 × 감 밀도
# ═══════════════════════════════════════

cluster_colors = {0: '#FF6B35', 1: '#2196F3', 2: '#4CAF50', 3: '#9C27B0', 4: '#F44336'}

fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# 1. 상극축 vs 감 밀도 (핵심 좌표계)
ax = axes[0, 0]
for c in range(5):
    sub = df[df['cluster'] == c]
    ax.scatter(sub['antagonism'], sub['gam_density'] * 100,
              s=3, alpha=0.3, c=cluster_colors[c], label=f'C{c} (n={len(sub)})')
ax.set_xlabel('Antagonism (상극축)', fontsize=12)
ax.set_ylabel('Gam Density (%)', fontsize=12)
ax.set_title('Protein Atlas: Antagonism vs Gam Density', fontsize=13, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.2)

# 2. 상극축 vs 길이
ax = axes[0, 1]
for c in range(5):
    sub = df[df['cluster'] == c]
    ax.scatter(sub['antagonism'], sub['length'],
              s=3, alpha=0.3, c=cluster_colors[c])
ax.set_xlabel('Antagonism', fontsize=12)
ax.set_ylabel('Length', fontsize=12)
ax.set_title('Antagonism vs Length', fontsize=13)
ax.set_yscale('log')
ax.grid(True, alpha=0.2)

# 3. 곤 비율 vs 건 비율 (색: 감 밀도)
ax = axes[1, 0]
sc = ax.scatter(df['gon_ratio'], df['geon_ratio'],
               s=3, alpha=0.3, c=df['gam_density'] * 100, cmap='YlOrRd')
plt.colorbar(sc, ax=ax, label='Gam Density (%)')
ax.set_xlabel('Gon Ratio', fontsize=12)
ax.set_ylabel('Geon Ratio', fontsize=12)
ax.set_title('Gon vs Geon (color: Gam Density)', fontsize=13)
ax.plot([0, 0.6], [0.6, 0], 'k--', alpha=0.3)  # 대각선
ax.grid(True, alpha=0.2)

# 4. 감 간격 히스토그램 (피보나치 표시)
ax = axes[1, 1]
all_gaps = []
for r in results:
    seq = None
    for s in sequences:
        if s['entity_id'] == r['entity_id']:
            seq = s['sequence']
            break
    if seq:
        gam_pos = [i for i, aa in enumerate(seq) if get_type(aa) == 'Gam']
        gaps = [gam_pos[i] - gam_pos[i-1] for i in range(1, len(gam_pos))]
        all_gaps.extend(gaps)

if all_gaps:
    gap_counter = Counter(all_gaps)
    max_gap = min(30, max(all_gaps))
    gaps_range = range(1, max_gap + 1)
    counts = [gap_counter.get(g, 0) for g in gaps_range]
    
    colors_bar = ['#FF9800' if g in FIB_SET else '#BBBBBB' for g in gaps_range]
    ax.bar(gaps_range, counts, color=colors_bar, edgecolor='none')
    ax.set_xlabel('Gam-Gam Gap', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.set_title('Gam-Gam Interval Distribution\n(Orange = Fibonacci)', fontsize=13)
    
    # 피보나치 표시
    for f in [1, 2, 3, 5, 8, 13, 21]:
        if f <= max_gap:
            ax.axvline(x=f, color='#FF6B35', linestyle='--', alpha=0.5)

plt.suptitle(f'UFoE Protein Atlas — {len(df)} Proteins Mapped', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('ufoe_protein_atlas.png', dpi=150, bbox_inches='tight')
plt.show()
print("💾 ufoe_protein_atlas.png")


## 7. 3D 인터랙티브 지도

In [ ]:
import plotly.express as px

# 3D 산점도: 상극축 × 감 밀도 × 길이
fig_3d = px.scatter_3d(
    df, 
    x='antagonism', 
    y='gam_density', 
    z='length',
    color='cluster',
    color_discrete_sequence=['#FF6B35', '#2196F3', '#4CAF50', '#9C27B0', '#F44336'],
    opacity=0.4,
    size_max=3,
    title='UFoE Protein Atlas — 3D Coordinate System',
    labels={
        'antagonism': 'Antagonism (상극축)',
        'gam_density': 'Gam Density (감 밀도)',
        'length': 'Length (길이)',
        'cluster': 'Cluster'
    },
)

fig_3d.update_traces(marker_size=2)
fig_3d.update_layout(
    scene=dict(
        xaxis_title='Antagonism',
        yaxis_title='Gam Density', 
        zaxis_title='Length (log)',
    ),
    width=900, height=700,
)

fig_3d.show()


## 8. 결과 저장

In [ ]:
# CSV 저장
df.to_csv('ufoe_protein_atlas.csv', index=False)
print(f"💾 ufoe_protein_atlas.csv ({len(df)} rows)")

# 클러스터 요약
summary = df.groupby('cluster').agg({
    'length': ['mean', 'std', 'count'],
    'antagonism': 'mean',
    'gam_density': 'mean',
    'mean_gap': 'mean',
    'near_fibonacci': 'mean',
    'gon_ratio': 'mean',
    'geon_ratio': 'mean',
    'g_ratio': 'mean',
    'p_ratio': 'mean',
    'c_ratio': 'mean',
}).round(4)

print("\n" + "=" * 70)
print("  UFoE Protein Atlas — Cluster Summary")
print("=" * 70)
print(summary.to_string())

# 피보나치 통계
if all_gaps:
    total = len(all_gaps)
    fib_exact = sum(1 for g in all_gaps if g in FIB_SET)
    fib_near = sum(1 for g in all_gaps if nearest_fibonacci(g)[1] <= 1)
    print(f"\n  전체 감-감 간격: {total}")
    print(f"  피보나치 정확: {fib_exact}/{total} = {fib_exact/total:.1%}")
    print(f"  피보나치 ±1: {fib_near}/{total} = {fib_near/total:.1%}")

print(f"\n{'═' * 70}")
print(f"  🧬 UFoE Protein Atlas 완성")
print(f"  {len(df)} 단백질 × 12칸 매트릭스 × 감 결절점 패턴")
print(f"  5 클러스터 = 건곤감리 + 중심")
print(f"{'═' * 70}")
